In [0]:
# not recommended, since it's slow
!pip install --upgrade tensorflow-gpu

In [0]:
import tensorflow as tf
print(tf.__version__)
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
!cp drive/My\ Drive/3DprintedDetection/rendered_transp_512_16_8_1.tar.gz ./
!sha256sum rendered_transp_512_16_8_1.tar.gz
!tar xf rendered_transp_512_16_8_1.tar.gz

In [0]:
!rm -rf images_croped

In [0]:
# train_images/crop_img.py
# -------------------------------

import os
import cv2
import numpy as np


def process(in_path: str, out_path: str, colors=None):
    img = cv2.imread(in_path, cv2.IMREAD_UNCHANGED)
    assert img.shape == (512, 512, 4)

    img, mask = crop_img(img)
    save_imgs(img, mask, out_path, colors)


def crop_img(img, blowout=True):
    mask = np.zeros((512,) * 2, dtype='uint8')
    mask[np.where(img[:, :, 3] != 0)] = 255

    if blowout:
        t_img = cv2.dilate(mask.copy(), np.ones((4, 4), np.uint8), iterations=1)

    _, contours, _ = cv2.findContours(t_img.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Find object with the biggest bounding box
    mx = (0, 0, 0, 0)      # biggest bounding box so far
    mx_area = 0
    org_area = img.shape[0] * img.shape[1]  # area if original image

    for i, cont in enumerate(contours):
        x, y, w, h = cv2.boundingRect(cont)
        area = w * h
        if area > mx_area and area < org_area:
            mx = x, y, w, h
            mx_area = area

    # make bounding box quadratic
    x, y, w, h = mx
    if w < h:
        x -= (h - w) // 2
        w = h
    elif h < w:
        y -= (w - h) // 2
        h = w

    assert x > 0
    assert y > 0
    assert x + w < img.shape[1]
    assert y + h < img.shape[0]

    return img[y:y + h, x:x + w], mask[y:y + h, x:x + w]


def resize_img(img, shape=(256, 256), debug=False):
    # resize image
    if debug and (img.shape[0] < shape[0] or img.shape[1] < shape[1]):
        print("Image shape is small: ", img.shape)
    return cv2.resize(img, shape)


def save_imgs(img, mask, out_path: str, colors=None):
    if not colors:
        img = resize_img(img[:, :, :3], shape=(128, 128))
        cv2.imwrite(out_path, img)
        return

    img = img[:, :, :3]
    img = (img * np.random.uniform(low=0.85, high=1.0)).astype('uint8')
    yellow_off = np.random.randint(0, 17)
    id0, id1 = np.where(img[:, :, 0] > yellow_off)
    img[id0, id1, 0] -= yellow_off

    h, w = img.shape[:2]
    mask = np.dstack((mask, mask, mask))
    mask_inverse = np.ones(mask.shape, dtype='uint8') * 255 - mask

    for j, col in enumerate(colors):
        backg = np.zeros((h, w, 3), dtype=np.uint8)
        backg[:, :] = col
        max_diffuse = np.random.randint(10, 41, dtype='uint8')
        backg += np.random.randint(0, max_diffuse,
                                   size=backg.shape,
                                   dtype='uint8')

        img_col = cv2.bitwise_and(img, mask)
        img_col += cv2.bitwise_and(backg, mask_inverse)
        img_col = resize_img(img_col, shape=(128, 128))

        out_p = "{}_col{}{}".format(out_path[:-4], j, out_path[-4:])
        cv2.imwrite(out_p, img_col)


if __name__ == "__main__":
    color_list = [[230, 10, 10], [10, 230, 10], [10, 10, 230], [150, 150, 150]]
    color_list += [[0, 0, 0], [200, 200, 10], [10, 200, 200], [200, 10, 200]]

    save_path = './images_croped'
    if not os.path.isdir(save_path):
        os.mkdir(save_path)

    path = './images'
    subfolders = os.listdir(path)
    failed_objects = []
    for i, folder in enumerate(subfolders):
        msg = 'Fortschritt - Ordner: {} von {}\r'.format(i, len(subfolders))
        print(msg)

        if not os.path.isdir('{}/{}'.format(path, folder)):
            continue

        if not os.path.isdir('%s/%s' % (save_path, folder)):
            os.mkdir('%s/%s' % (save_path, folder))

        content = os.listdir('{}/{}'.format(path, folder))
        for img_name in content:
            input_path = '%s/%s/%s' % (path, folder, img_name)
            output_path = '%s/%s/%s' % (save_path, folder, img_name)
            try:
                process(input_path, output_path, color_list)
            except AssertionError:
                if folder not in failed_objects:
                    failed_objects.append(folder)
    print('\n', failed_objects)

In [0]:
# prediction/calculate_mean_std_batchwise.py
# -------------------------------

import os

cnt = 0
PATH = './images_croped/'

subfolders = os.listdir(PATH)
subfolders = [k for k in subfolders if os.path.isdir('{}{}'.format(PATH, k))]

for folder in subfolders:
  nb = len([k for k in os.listdir('{}{}'.format(PATH, folder))])
  cnt += nb

print(cnt)

In [0]:
# prediction/calculate_mean_std_batchwise.py
# -------------------------------

import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator

TRAIN_PATH = './images_croped/'
IMG_WIDTH = 128
BATCH_SIZE = 2048

datagen = ImageDataGenerator(rescale=1. / 255)

gen = datagen.flow_from_directory(
    directory=TRAIN_PATH,
    target_size=(IMG_WIDTH, IMG_WIDTH),
    color_mode="rgb",
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    shuffle=False
)

batches = 88
assert float(batches) == cnt / BATCH_SIZE

print("Calculate mean")

mean = np.zeros((IMG_WIDTH, IMG_WIDTH, 3))
for i in range(batches):
    X, _ = gen.next()
    mean += np.mean(X, axis=0)
    if i % 10 == 0:
        print(i)
mean /= batches
print(mean.shape)

print("Calculate std ")

var = np.zeros((IMG_WIDTH, IMG_WIDTH, 3))
for i in range(batches):
    X, _ = gen.next()
    var += np.sum((X - mean) ** 2, axis=0)
    if i % 10 == 0:
        print(i)
var /= (cnt - 1)
std = np.sqrt(var)
print(std.shape)

np.save('featurewise_mean', mean)
np.save('featurewise_std', std)

In [0]:
# prediction/calculate_mean_std_batchwise.py
# -------------------------------

!cp featurewise_std.npy drive/My\ Drive/ba_classification
!cp featurewise_mean.npy drive/My\ Drive/ba_classification

In [0]:
# copy std and mean from Gdrive
!cp drive/My\ Drive/3DprintedDetection/cnn_classification/featurewise_std.npy ./
!cp drive/My\ Drive/3DprintedDetection/cnn_classification/featurewise_mean.npy ./

In [0]:
# prediction/cnn_datagenerator.py
# -------------------------------

from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import warnings


class MyImageDataGen(ImageDataGenerator):
    """ My own ImageDataGenerator which inherit from the original
    and applies precalculated std and mean"""

    def __init__(self, mean=None, std=None, **kwargs):
        super().__init__(**kwargs)
        self._mean = mean
        self._std = std

    def standardize(self, x):
        if self._mean is not None and self._std is not None:
            assert x.shape[-3:] == self._mean.shape
            assert x.shape[-3:] == self._std.shape

        if self.preprocessing_function:
            x = self.preprocessing_function(x)
        if self.rescale:
            x *= self.rescale
        if self.samplewise_center:
            x -= np.mean(x, keepdims=True)
        if self.samplewise_std_normalization:
            x /= (np.std(x, keepdims=True) + 1e-6)

        if self.featurewise_center:
            if self._mean is not None:
                x -= self._mean
            else:
                warnings.warn('This MyImageDataGen specifies '
                              '`featurewise_center`, but the mean isn\'t given.')
        if self.featurewise_std_normalization:
            if self._std is not None:
                x /= (self._std + 1e-6)
            else:
                warnings.warn('This MyImageDataGen specifies '
                              '`featurewise_std_normalization`, '
                              'but the std isn\'t given.')
        if self.zca_whitening:
            warnings.warn('This MyImageDataGen specifies '
                          '`zca_whitening`, but isn\'t implemented.')
        return x


def get_train_gen(mean_path='./featurewise_mean.npy', std_path='./featurewise_std.npy'):
    mean = np.load(mean_path, allow_pickle=True)
    std = np.load(std_path, allow_pickle=True)
    kwargs = dict(
        # set rescaling factor (applied before any other transformation)
        rescale=1. / 255,
        # set input mean to 0 over the dataset
        featurewise_center=True,
        featurewise_std_normalization=True,
        # samplewise_center=True,
        # samplewise_std_normalization=True,

        # randomly rotate images in the range (degrees, 0 to 180)
        # rotation_range=10,
        # randomly shift images horizontally (fraction of total width)
        width_shift_range=0.02,
        # randomly shift images vertically (fraction of total height)
        height_shift_range=0.02,
        # horizontal_flip=True,  # randomly flip images
        # vertical_flip=True,  # randomly flip images
    )
    datagen = MyImageDataGen(mean, std, **kwargs)
    return datagen


def get_test_gen(mean_path='./featurewise_mean.npy', std_path='./featurewise_std.npy'):
    mean = np.load(mean_path, allow_pickle=True)
    std = np.load(std_path, allow_pickle=True)
    kwargs = dict(
        # set rescaling factor (applied before any other transformation)
        rescale=1. / 255,
        # set input mean to 0 over the dataset
        featurewise_center=True,
        featurewise_std_normalization=True,
        # samplewise_center=True,
        # samplewise_std_normalization=True,
    )
    datagen = MyImageDataGen(mean, std, **kwargs)
    return datagen

In [0]:
# Tensorflow 2 needed
!rm -rf logs
%load_ext tensorboard

In [0]:
%reload_ext tensorboard
%tensorboard --logdir logs/fit

In [0]:
# prediction/cnn_pretrained.py
# -------------------------------

import os
import matplotlib.pyplot as plt
import tensorflow.keras as keras
from tensorflow.keras import layers
from tensorflow.keras.models import Model
# from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.applications.vgg16 import VGG16

### --- paths --- ###

TRAIN_PATH = './images_croped'
MODEL_JSON = './cnn_vgg16.json'
MODEL_WEIGHTS = './cnn_vgg16.hdf5'
# LOG_DIR = "logs/fit"

### --- paths end --- ###

### --- parameters --- ###

TRAIN_CONV = True

img_width = 128
num_classes = 45
nb_train_samples = 180224

reg = 1e-3

epochs = 8
batch_size = 64
learning_rate = 1e-4

### --- parameters end --- ###

### --- model --- ###

if os.path.isfile(MODEL_JSON) and os.path.isfile(MODEL_WEIGHTS):
    print('Load model from file')
    with open(MODEL_JSON, 'r') as file:
        json_string = file.readline()
    model = keras.models.model_from_json(json_string)
    model.load_weights(MODEL_WEIGHTS)
else:
    print('Build new model')
    # create the base pre-trained model
    base_model = VGG16(include_top=False,
                       input_shape=(img_width, img_width, 3))

    if TRAIN_CONV:
        # get output Block 4
        x = base_model.get_layer('block4_conv2').output

        # last convolutional layers
        x = layers.Conv2D(512, (3, 3),
                        activation='relu',
                        padding='same',
                        name='block4_conv3')(x)
        x = layers.MaxPooling2D((2, 2), strides=(2, 2),
                                name='block4_pool')(x)
    else:
        # get output Block 4
        x = base_model.get_layer('block4_pool').output

    # classification block
    x = layers.Flatten(name='flatten')(x)

    x = layers.Dense(4096, activation='relu', name='fc1',
                    kernel_regularizer=keras.regularizers.l2(reg))(x)
    x = layers.Dense(4096, activation='relu', name='fc2',
                    kernel_regularizer=keras.regularizers.l2(reg))(x)

    predictions = layers.Dense(num_classes, activation='softmax',
                               name='predictions')(x)

    model = Model(inputs=base_model.input, outputs=predictions)

    for layer in base_model.layers:
        layer.trainable = False

if TRAIN_CONV:
    assert not model.get_layer(name='block4_conv2').trainable
    assert model.get_layer(name='block4_conv3').trainable
else:
    assert not model.get_layer(name='block4_conv3').trainable
    assert model.get_layer(name='fc1').trainable

### --- end --- ###

### --- training --- ###

# initiate adam optimizer
opt = keras.optimizers.Adam(lr=learning_rate)

model.compile(
    optimizer=opt,
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

datagen = get_train_gen().flow_from_directory(
    directory=TRAIN_PATH,
    target_size=(img_width, img_width),
    color_mode="rgb",
    batch_size=batch_size,
    class_mode="categorical",
    shuffle=True
)

# tfboard = TensorBoard(log_dir=LOG_DIR, histogram_freq=1)

# Fit the model on the batches generated by datagen
history = model.fit_generator(
    generator=datagen,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    workers=2,
    verbose=1 #,
    # callbacks=[tfboard]
)

### --- end training --- ###

### --- save model --- ###

model.summary()

json_string = model.to_json()
with open(MODEL_JSON, 'w') as file:
    file.write(json_string + '\n')
model.save_weights(MODEL_WEIGHTS)

### --- end save --- ###

In [0]:
plt.style.use('ggplot')
plt.figure(figsize=(12,5))

plt.subplot(1,2,1)
plt.plot(history.history['acc'])
plt.ylabel('accuracy')
plt.xlabel('epoch')

plt.subplot(1,2,2)
plt.plot(history.history['loss'])
plt.ylabel('loss')
plt.xlabel('epoch')

plt.savefig('./cnn_vgg16_train.png')
plt.show()
plt.close()

In [0]:
!cp cnn_vgg16.json drive/My\ Drive/ba_classification
!cp cnn_vgg16.hdf5 drive/My\ Drive/ba_classification
!cp cnn_vgg16_train.png drive/My\ Drive/ba_classification

In [0]:
# prediction/cnn_own.py
# -------------------------------

import os
import matplotlib.pyplot as plt
import tensorflow.keras as keras
from tensorflow.keras.layers import Activation, Conv2D, Dense
from tensorflow.keras.layers import Dropout, Flatten, MaxPooling2D

### --- paths --- ###

TRAIN_PATH = './images_croped'
MODEL_JSON = './cnn_own.json'
MODEL_WEIGHTS = './cnn_own.hdf5'

### --- paths end --- ###

### --- parameters --- ###

img_width = 128
num_classes = 45
nb_train_samples = 180224

reg = 1e-3

epochs = 16
batch_size = 64
learning_rate = 1e-4

### --- parameters end --- ###

### --- define model --- ###

if os.path.isfile(MODEL_JSON) and os.path.isfile(MODEL_WEIGHTS):
    print('Load model from file')
    with open(MODEL_JSON, 'r') as file:
        json_string = file.readline()
    model = keras.models.model_from_json(json_string)
    model.load_weights(MODEL_WEIGHTS)
else:
    print('Build new model')
    model = keras.Sequential()

    # ConvLayer 1 - Input
    model.add(Conv2D(
        32, 3,
        # strides=(2, 2),
        padding='same',
        input_shape=(img_width, img_width, 3),
        kernel_regularizer=keras.regularizers.l2(reg)
    ))
    model.add(Activation('relu'))

    # ConvLayer 2
    model.add(Conv2D(
        32, 3,
        padding='same',
        kernel_regularizer=keras.regularizers.l2(reg)
    ))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    # ConvLayer 3
    model.add(Conv2D(
        64, 3,
        padding='same',
        kernel_regularizer=keras.regularizers.l2(reg)
    ))
    model.add(Activation('relu'))

    # ConvLayer 4
    model.add(Conv2D(
        64, 3,
        padding='same',
        kernel_regularizer=keras.regularizers.l2(reg)
    ))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    # ConvLayer 5
    model.add(Conv2D(
        128, 3,
        padding='same',
        kernel_regularizer=keras.regularizers.l2(reg)
    ))
    model.add(Activation('relu'))

    # ConvLayer 6
    model.add(Conv2D(
        128, 3,
        padding='same',
        kernel_regularizer=keras.regularizers.l2(reg)
    ))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    # Flat
    model.add(Flatten())

    # Fully Conected Layer 7
    model.add(Dense(2048, kernel_regularizer=keras.regularizers.l2(reg)))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))

    # Fully Conected Layer 8

    model.add(Dense(2048, kernel_regularizer=keras.regularizers.l2(reg)))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))

    # Fully Connected Layer 9 - Output
    model.add(Dense(num_classes, kernel_regularizer=keras.regularizers.l2(reg)))
    model.add(Activation('softmax'))

### --- end definition --- ###

### --- training --- ###

# initiate adam optimizer
opt = keras.optimizers.Adam(lr=learning_rate)

model.compile(
    optimizer=opt,
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

datagen = get_train_gen().flow_from_directory(
    directory=TRAIN_PATH,
    target_size=(img_width, img_width),
    color_mode="rgb",
    batch_size=batch_size,
    class_mode="categorical",
    shuffle=True
)

# Fit the model on the batches generated by datagen
history = model.fit_generator(
    generator=datagen,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    workers=2,
    verbose=1
)

### --- end training --- ###

### --- save model --- ###

model.summary()

json_string = model.to_json()
with open(MODEL_JSON, 'w') as file:
    file.write(json_string + '\n')
model.save_weights(MODEL_WEIGHTS)

### --- end save --- ###

### --- show learning --- ###

In [0]:
plt.style.use('ggplot')
plt.figure(figsize=(12,5))

plt.subplot(1,2,1)
plt.plot(history.history['acc'])
plt.ylabel('accuracy')
plt.xlabel('epoch')

plt.subplot(1,2,2)
plt.plot(history.history['loss'])
plt.ylabel('loss')
plt.xlabel('epoch')

plt.savefig('./cnn_own_train.png')
plt.show()
plt.close()

In [0]:
!cp cnn_own.json drive/My\ Drive/ba_classification
!cp cnn_own.hdf5 drive/My\ Drive/ba_classification
!cp cnn_own_train.png drive/My\ Drive/ba_classification

In [0]:
# rgb vs. hsv visualization
# taken from https://realpython.com/python-opencv-color-spaces/

import cv2
import numpy as np
from matplotlib import cm
from matplotlib import colors
from google.colab import files
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

plt.rcParams['figure.figsize'] = [10, 10]

uploaded = files.upload()
fname = list(uploaded.keys())[0]

nemo = cv2.imread(fname, 1)
if nemo.shape[0] > 512 or nemo.shape[1] > 512:
    nemo = cv2.resize(nemo, (512, 512))

nemo = cv2.cvtColor(nemo, cv2.COLOR_BGR2RGB)
r, g, b = cv2.split(nemo)
fig = plt.figure()
axis = fig.add_subplot(1, 1, 1, projection="3d")

pixel_colors = nemo.reshape((np.shape(nemo)[0]*np.shape(nemo)[1], 3))
norm = colors.Normalize(vmin=-1.,vmax=1.)
norm.autoscale(pixel_colors)
pixel_colors = norm(pixel_colors).tolist()

axis.scatter(r.flatten(), g.flatten(), b.flatten(), facecolors=pixel_colors, marker=".")
axis.set_xlabel("Red")
axis.set_ylabel("Green")
axis.set_zlabel("Blue")
plt.title('RGB-Farbraum')
plt.savefig('rgb_viz.png')
plt.show()

hsv_nemo = cv2.cvtColor(nemo, cv2.COLOR_RGB2HSV)

h, s, v = cv2.split(hsv_nemo)
fig = plt.figure()
axis = fig.add_subplot(1, 1, 1, projection="3d")

axis.scatter(h.flatten(), s.flatten(), v.flatten(), facecolors=pixel_colors, marker=".")
axis.set_xlabel("Hue")
axis.set_ylabel("Saturation")
axis.set_zlabel("Value")
plt.title('HSV-Farbraum')
plt.savefig('hsv_viz.png')
plt.show()